<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2024 Daniela Concha. Todos los derechos reservados.</font>
</p>

# Tabla de contenidos

1. [Ejemplo consumo de `*args` y `**kwargs` en MRO](#Ejemplo_consumo_de_*args_y_**kwargs_en_MRO)
    1. [No se utiliza `super()`](#No_se_utiliza_super())
    2. [Sí se utiliza `super()`](#Sí_se_utiliza_super())
2. [Ejemplo atributos por defecto](#Ejemplo_atributos_por_defecto)
    1. [Entregar instancia vs `None`](#Entregar_instancia_vs_None)
3. [Ejemplo desempaquetamiento](#Ejemplo_desempaquetamiento)
    1. [Desempaquetar con `*` y `**`](#Desempaquetar_con_*_y_**)

## Ejemplo consumo de `*args` y `**kwargs` en MRO

In [1]:
def print_clase_atributos(clase: str = '', atributos: tuple = ()) -> None:
    '''
    Función que recibe la información de una clase (Nombre, atributos, args, kwargs)
    y lo imprime en un formato en específico
    '''
    platilla_texto = '{:<15} | {:<15} | {:<15} | {}'
    if not (clase and atributos):
        print(platilla_texto.format('Nombre', 'Atributo Input', '*args', '**kwargs'))
        print('-' * 100)
    else:
        print(platilla_texto.format(clase, atributos[0], repr(atributos[1]), repr(atributos[2])))

### No se utiliza `super()`

In [2]:
class ClaseAbuelo:
    def __init__(self, atributo3, *args, **kwargs) -> None:
        print_clase_atributos('ClaseAbuelo', (atributo3, args, kwargs))
        self.atributo3 = atributo3


class ClasePadre(ClaseAbuelo):
    def __init__(self, atributo1, *args, **kwargs) -> None:
        print_clase_atributos('ClasePadre', (atributo1, args, kwargs))
        self.atributo1 = atributo1
        ClaseAbuelo.__init__(self, *args, **kwargs)


class ClaseMadre(ClaseAbuelo):
    def __init__(self, atributo2, *args, **kwargs) -> None:
        print_clase_atributos('ClaseMadre', (atributo2, args, kwargs))
        self.atributo2 = atributo2
        ClaseAbuelo.__init__(self, *args, **kwargs)


class ClaseHija(ClasePadre, ClaseMadre):
    def __init__(self, atributo1, atributo2, atributo3) -> None:
        print_clase_atributos('ClaseHija', (f'{atributo1}, {atributo2}, {atributo3}', '-', '-'))
        ClasePadre.__init__(self, atributo1, atributo3)
        ClaseMadre.__init__(self, atributo2, atributo3)


if __name__ == '__main__':
    # Cuando no se hace uso de 'super()' para manejar la multi-herencia, 
    # el recorrido del MRO pasará 2 veces por la ClaseAbuelo.

    # Para verlo con mayor claridad, revisar el flujo que se muestra en
    # la presentación de la clase 4.

    print_clase_atributos()
    ClaseHija(1, 2, 3)

Nombre          | Atributo Input  | *args           | **kwargs
----------------------------------------------------------------------------------------------------
ClaseHija       | 1, 2, 3         | '-'             | '-'
ClasePadre      | 1               | (3,)            | {}
ClaseAbuelo     | 3               | ()              | {}
ClaseMadre      | 2               | (3,)            | {}
ClaseAbuelo     | 3               | ()              | {}


### Sí se utiliza `super()`

In [3]:
class ClaseAbuelo:
    def __init__(self, atributo3, *args, **kwargs) -> None:
        print_clase_atributos('ClaseAbuelo', (atributo3, args, kwargs))
        self.atributo3 = atributo3


class ClasePadre(ClaseAbuelo):
    def __init__(self, atributo1, *args, **kwargs) -> None:
        print_clase_atributos('ClasePadre', (atributo1, args, kwargs))
        self.atributo1 = atributo1
        super().__init__(*args, **kwargs)


class ClaseMadre(ClaseAbuelo):
    def __init__(self, atributo2, *args, **kwargs) -> None:
        print_clase_atributos('ClaseMadre', (atributo2, args, kwargs))
        self.atributo2 = atributo2
        super().__init__(*args, **kwargs)


class ClaseHija(ClasePadre, ClaseMadre):
    def __init__(self, *args, **kwargs) -> None:
        print_clase_atributos('ClaseHija', ('', args, kwargs))
        super().__init__(*args, **kwargs)


if __name__ == '__main__':
    # Cuando sí se hace uso de 'super()' para manejar la multi-herencia, 
    # el recorrido del MRO pasará primero por las ClasePadre y ClaseMadre,
    # después por la ClaseAbuelo.

    # Para verlo con mayor claridad, revisar el flujo que se muestra en
    # la presentación de la clase 4.

    print_clase_atributos()
    ClaseHija(1, 2, 3)
    print()

    print_clase_atributos()
    ClaseHija(atributo2=2, atributo1=1, atributo3=3)

Nombre          | Atributo Input  | *args           | **kwargs
----------------------------------------------------------------------------------------------------
ClaseHija       |                 | (1, 2, 3)       | {}
ClasePadre      | 1               | (2, 3)          | {}
ClaseMadre      | 2               | (3,)            | {}
ClaseAbuelo     | 3               | ()              | {}

Nombre          | Atributo Input  | *args           | **kwargs
----------------------------------------------------------------------------------------------------
ClaseHija       |                 | ()              | {'atributo2': 2, 'atributo1': 1, 'atributo3': 3}
ClasePadre      | 1               | ()              | {'atributo2': 2, 'atributo3': 3}
ClaseMadre      | 2               | ()              | {'atributo3': 3}
ClaseAbuelo     | 3               | ()              | {}


## Ejemplo atributos por defecto
### Entregar instancia vs `None`

In [4]:
def función_mala(lista: list = []) -> None:
    '''
    Función que recibe una lista.
    Le agrega a dicha lista el valor de su largo y la imprime.

    Si no recibe una lista, usará la misma lista por defecto
    en los llamados de la función.
    '''
    lista.append(len(lista))
    print(lista)


def función_buena(lista: list | None = None) -> None:
    '''
    Función que recibe una lista.
    Le agrega a dicha lista el valor de su largo y la imprime.
    
    Si no recibe una lista, creará una instancia distinta de
    lista en los llamados de la función.
    '''
    if lista is None:
        lista = []

    lista.append(len(lista))
    print(lista)


if __name__ == '__main__':
    print('Ejecución usando la función mala')
    función_mala(['a', 'b'])
    función_mala()
    función_mala()
    función_mala()

    print('\nEjecución usando la función buena')
    función_buena(['a', 'b'])
    función_buena()
    función_buena()
    función_buena()

Ejecución usando la función mala
['a', 'b', 2]
[0]
[0, 1]
[0, 1, 2]

Ejecución usando la función buena
['a', 'b', 2]
[0]
[0]
[0]


## Ejemplo desempaquetamiento
### Desempaquetar con `*` y `**`

In [5]:
# Al momento de trabajar con estructuras que se pueden desempaquetar,
# como listas, tuplas y diccionarios
lista = [1, 2, 3, 4, 5, 6, 7]
diccionario = {'a': 1, 'b': 2, 'c': 3}

# Podemos utilizar el operador * para desempaquetar la información de estas estructuras:
# (Los * y ** no solo se utilizan en la definición de clases y funciones)
print('Podemos desempaquetar:')
print('- El contenido de listas:        ', *lista)
print('- Las llaves de un diccionario:  ', *diccionario)
print('- Los valores de un diccionario: ', *diccionario.values())
print('- Los pares de un diccionario:   ', *diccionario.items())

# Podemos utilizar el operador ** en contextos donde se deba asociar un valor a una llave.
# Por ejemplo, cuando usamos strings y format:
print('- En textos que usan format:      {a} {c} {a} {b}'.format(**diccionario))

Podemos desempaquetar:
- El contenido de listas:         1 2 3 4 5 6 7
- Las llaves de un diccionario:   a b c
- Los valores de un diccionario:  1 2 3
- Los pares de un diccionario:    ('a', 1) ('b', 2) ('c', 3)
- En textos que usan format:      1 3 1 2
